In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
spark.conf.set("spark.sql.shuffle.partiotins",6)

In [4]:
df = spark.read.format("json").load("/home/jovyan/work/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [5]:
df.sort("count").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [6]:
df.orderBy("count","DEST_COUNTRY_NAME").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [7]:
from pyspark.sql.functions import expr
df.orderBy(expr("count desc"),expr("DEST_COUNTRY_NAME desc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [8]:
df.orderBy(expr("count desc"),expr("DEST_COUNTRY_NAME asc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [13]:
df.limit(2).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+



In [10]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   spark-5-structured-operations-py.ipynb
	modified:   spark-5-structured-operations-scala.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [12]:
%%! git add .
git status
git commit -m "spark notebooks"
git push origin master

['On branch master',
 'Changes not staged for commit:',
 '  (use "git add <file>..." to update what will be committed)',
 '  (use "git checkout -- <file>..." to discard changes in working directory)',
 '',
 '\tmodified:   spark-5-structured-operations-py.ipynb',
 '\tmodified:   spark-5-structured-operations-scala.ipynb',
 '',
 'no changes added to commit (use "git add" and/or "git commit -a")',
 'On branch master',
 'Changes not staged for commit:',
 '\tmodified:   spark-5-structured-operations-py.ipynb',
 '\tmodified:   spark-5-structured-operations-scala.ipynb',
 '',
 'no changes added to commit',
 'Everything up-to-date']